<header style="background-color: rgb(0, 62, 92); color: white; margin-top: 20px; padding:28px; ">
  <p style=" text-align: center; font-size: 42px;">   
   <strong> TOPOLOGICAL DATA ANALYSIS </strong></p>
  <p style=" text-align: center; font-size: 36px;"><strong> GROUP PROJECT (GROUP 10) </strong></p>
  <p style=" text-align: center; font-size: 28px;"> Fadhil Ramadhani  </p>
    <p style=" text-align: center; font-size: 28px;"> Evans MUSONDA </p>
    <p style=" text-align: center; font-size: 28px;"> Emmanuel DJIMMO TALLA </p>
  <p style=" text-align: center; font-size: 24px;"> African Institute for Mathematical Sciences </p>
</header>

# Project 1: Gauss Reduction Algorithm and Incremental Algorithm for Betti Numbers

In this project, we implement the **Gauss Reduction Algorithm** (Algorithm 9) to reduce the boundary matrix and the **Incremental Algorithm** (Algorithm 8) to compute **Betti numbers** $β_0(K), β_1(K), \dots, β_{\text{dim}(K)}(K)$ for a simplicial complex $K$. The project first focuses on reducing the boundary matrix using modulo-2 arithmetic to efficiently identify homology classes. Subsequently, we incrementally compute Betti numbers to quantify the number of connected components, loops, and voids in topological spaces such as the **circle ($S^1$)**, **2-dimensional sphere ($S^2$)**, and **torus ($T^2$)**.

The implementation is based on key intermediate functions. The `build_boundary_matrix` function constructs the boundary matrix to represent relationships between simplices and their faces, while the `reduce_boundary_matrix` function simplifies this matrix using column operations modulo-2. Additionally, the `ispositive` function identifies simplices that contribute positively to homology, enabling efficient computation of Betti numbers.

The algorithms are validated by applying the **Incremental Algorithm** to reduced boundary matrices constructed for the triangulations of $S^1$, $S^2$, and $T^2$. This demonstrates the effectiveness of combining matrix reduction with incremental computation in capturing the topological features of these spaces.


In [1]:
# Import neccessary libraries

import gudhi 
import numpy as np
import networkx as nx

In [2]:
def build_boundary_matrix(simplices):
    """
    Constructs the boundary matrix for the given simplices.
    :param simplices: List of simplices (as lists of vertices).
    :return: Boundary matrix as a numpy array.
    """
    n = len(simplices)
    boundary_matrix = np.zeros((n, n), dtype=int)
    simplex_index = {tuple(simplices[i]): i for i in range(n)}

    for j in range(n):
        simplex = simplices[j]
        if len(simplex) > 1:  # Only consider higher-dimensional simplices
            for i in range(len(simplex)):
                face = tuple(simplex[:i] + simplex[i + 1:])
                if face in simplex_index:
                    boundary_matrix[simplex_index[face], j] = 1

    return boundary_matrix

In [3]:
def ispositive(matrix):
    """
    Determines whether each column in the reduced boundary matrix is positive.
    :param matrix: Reduced boundary matrix.
    :return: Positivity vector.
    """
    n = matrix.shape[1]
    sigmas = np.ones(n, dtype=int)
    for j in range(n):
        if np.any(matrix[:, j] == 1):
            sigmas[j] = 0
    return sigmas

def extract_simplices_from_simplextree(simplex_tree):
    """
    Extracts all simplices from a Gudhi SimplexTree.
    :param simplex_tree: Gudhi SimplexTree object.
    :return: List of simplices (as lists of vertices).
    """
    return [list(simplex[0]) for simplex in simplex_tree.get_simplices()]

After defining the foundational functions `build_boundary_matrix`, `ispositive`, and `extract_simplices_from_simplextree`, we now proceed to implement the **Gauss Reduction Algorithm**. This algorithm simplifies the boundary matrix by applying modulo-2 operations, enabling the identification of pivotal simplices and preparing the data for efficient computation of homology contributions.


In [4]:
def reduce_boundary_matrix(D):
    """
    Reduces a boundary matrix using column operations modulo 2.
    :param D: Boundary matrix (numpy array).
    :return: Reduced boundary matrix.
    """
    def phi(column):
        nonzero_indices = np.where(column != 0)[0]
        return nonzero_indices[-1] if nonzero_indices.size > 0 else None

    n = D.shape[1]
    for j in range(n):
        while True:
            phi_j = phi(D[:, j])
            if phi_j is None:
                break

            found = False
            for i in range(j):
                if phi(D[:, i]) == phi_j:
                    D[:, j] = (D[:, j] + D[:, i]) % 2
                    found = True
                    break

            if not found:
                break

    return D

With the **Gauss Reduction Algorithm** implemented, we now move to its application for reducing the boundary matrices of specific spaces. The simplicial complexes considered include triangulations of the **circle ($S^1$)**, **2-dimensional sphere ($S^2$)**, and **torus ($T^2$)**.

In [5]:
# We construct the Gudhi SimplexTree for a circle (S^1)

def circle_simplices():
    """
    Constructs the Gudhi SimplexTree for a circle (S^1).
    The triangulation includes three vertices and three edges forming a loop.
    """
    circle = gudhi.SimplexTree()
    circle.insert([0])
    circle.insert([1])
    circle.insert([2])
    circle.insert([0, 1])
    circle.insert([1, 2])
    circle.insert([2, 0])
    return extract_simplices_from_simplextree(circle)  # Extract simplices from SimplexTree




# We construct the Gudhi SimplexTree for a sphere (S^2)

def sphere_simplices():
    """
    Constructs the Gudhi SimplexTree for a sphere (S^2).
    The triangulation is based on a tetrahedron with four vertices,
    six edges, and four triangular faces forming a closed surface.
    """
    sphere = gudhi.SimplexTree()
    sphere.insert([0])
    sphere.insert([1])
    sphere.insert([2])
    sphere.insert([3])
    sphere.insert([0, 1])
    sphere.insert([1, 2])
    sphere.insert([2, 3])
    sphere.insert([3, 0])
    sphere.insert([0, 2])
    sphere.insert([1, 3])
    sphere.insert([0, 1, 2])
    sphere.insert([0, 1, 3])
    sphere.insert([0, 2, 3])
    sphere.insert([1, 2, 3])
    return extract_simplices_from_simplextree(sphere)  # Extract simplices from SimplexTree




# We construct the Gudhi SimplexTree for a torus (T^2)

def torus_simplices():
    """
    Constructs the Gudhi SimplexTree for a torus (T^2).
    Explicitly adds vertices, edges, and triangles.
    """
    torus = gudhi.SimplexTree()
    for vertex in range(9):  # Insert vertices 0 through 8
        torus.insert([vertex])
    torus.insert([3, 4])
    torus.insert([4, 8])
    torus.insert([3, 8])
    torus.insert([3, 7])
    torus.insert([7, 8])
    torus.insert([7, 6])
    torus.insert([8, 6])
    torus.insert([7, 5])
    torus.insert([5, 6])
    torus.insert([5, 4])
    torus.insert([6, 4])
    torus.insert([5, 3])
    torus.insert([0, 3])
    torus.insert([0, 7])
    torus.insert([2, 7])
    torus.insert([0, 2])
    torus.insert([1, 5])
    torus.insert([2, 1])
    torus.insert([4, 0])
    torus.insert([6, 1])
    torus.insert([8, 2])
    torus.insert([3, 4, 8])
    torus.insert([3, 7, 8])
    torus.insert([7, 8, 6])
    torus.insert([7, 5, 6])
    torus.insert([5, 6, 4])
    torus.insert([5, 4, 3])
    torus.insert([0, 3, 7])
    torus.insert([0, 2, 7])
    torus.insert([2, 7, 5])
    torus.insert([2, 1, 5])
    torus.insert([1, 5, 3])
    torus.insert([0, 1, 3])
    torus.insert([4, 0, 2])
    torus.insert([4, 2, 8])
    torus.insert([8, 2, 1])
    torus.insert([8, 1, 6])
    torus.insert([6, 1, 0])
    torus.insert([6, 4, 0])
    return extract_simplices_from_simplextree(torus)  # Extract simplices from SimplexTree


In [13]:
if __name__ == "__main__":
    
    # Example usage with the circle simplices
    print("Boundary Matrix for Circle:")
    circle = circle_simplices()
    print(build_boundary_matrix(circle))
    print("************************************")
    
    print("Reduced Boundary Matrix for Circle:")
    print(reduce_boundary_matrix(build_boundary_matrix(circle)))
    print("====================================")
    
    # Example usage with the sphere simplices
    print("Boundary Matrix for Sphere:")
    sphere = sphere_simplices()
    print(build_boundary_matrix(sphere))
    print("************************************")
    
    print("Reduced Boundary Matrix for Sphere:")
    print(reduce_boundary_matrix(build_boundary_matrix(sphere)))
    print("====================================")
    
    # Example usage with the torus simplices
    print("Boundary Matrix for Torus:")
    torus = torus_simplices()
    print(build_boundary_matrix(torus))
    print("************************************")
    
    print("Reduced Boundary Matrix for Torus:")
    print(reduce_boundary_matrix(build_boundary_matrix(torus)))


Boundary Matrix for Circle:
[[0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [1 1 0 0 0 0]
 [0 0 0 0 0 0]
 [1 0 0 1 0 0]
 [0 1 0 1 0 0]]
************************************
Reduced Boundary Matrix for Circle:
[[0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [1 1 0 0 0 0]
 [0 0 0 0 0 0]
 [1 0 0 0 0 0]
 [0 1 0 0 0 0]]
Boundary Matrix for Sphere:
[[0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [1 1 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [1 0 0 1 0 0 0 0 0 0 0 0 0 0]
 [0 1 0 1 0 0 0 0 0 0 0 0 0 0]
 [0 0 1 0 1 1 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 1 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 1 0 0 0 0 0 0]
 [0 0 1 0 0 0 0 0 1 1 0 0 0 0]
 [0 0 0 1 0 0 0 1 0 0 0 0 0 0]
 [0 0 0 0 1 0 0 0 1 0 0 1 0 0]
 [0 0 0 0 0 1 0 0 0 1 0 1 0 0]]
************************************
Reduced Boundary Matrix for Sphere:
[[0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [1 1 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [1 0 0 1 0 0 0 0 0 0 0 0 0 0]
 [0 1 0 1 0 0 0 0 0 0 0 0 

In [7]:
def incremental_algorithm(simplices):
    """
    Incremental Algorithm to compute Betti numbers.
    :param simplices: List of simplices (as lists of vertices).
    :return: List of Betti numbers.
    """
    # Determine the maximum dimension of the simplicial complex
    max_dimension = max(len(simplex) - 1 for simplex in simplices)

    # Initialize Betti numbers for all dimensions
    betti_numbers = [0] * (max_dimension + 1)

    # Sort simplices by dimension
    simplices = sorted(simplices, key=len)

    # Process simplices incrementally
    boundary_matrix = build_boundary_matrix(simplices)  # Build the boundary matrix
    reduced_matrix = reduce_boundary_matrix(boundary_matrix)  # Reduce the boundary matrix
    positivity_vector = ispositive(reduced_matrix)  # Determine positive simplices

    for i, simplex in enumerate(simplices):
        d = len(simplex) - 1  # Dimension of the current simplex

        # Check if the simplex is positive (creates a homology class)
        if positivity_vector[i] == 1:
            betti_numbers[d] += 1  # Positive case: Increment Betti number for dimension d
        else:
            # If not positive and dimension > 0 :  Decrement the Betti number for dimension d-1
            if d > 0:
                betti_numbers[d - 1] -= 1

    return betti_numbers

With the **Incremental Algorithm** defined, we now apply it to compute Betti numbers for specific topological spaces: the **circle ($S^1$)**, the **sphere ($S^2$)**, and the **torus ($T^2$)**. These examples illustrate the effectiveness of the algorithm in capturing the topological features of these spaces.

In [12]:
if __name__ == "__main__":
    
    # Print Betti Numbers for Circle (S^1)
    print("Betti Numbers for Circle (S^1):")
    print(incremental_algorithm(circle_simplices()))

    # Print Betti Numbers for Sphere (S^2)
    print("\nBetti Numbers for Sphere (S^2):")
    print(incremental_algorithm(sphere_simplices()))

    # Print Betti Numbers for Torus (T^2)
    print("\nBetti Numbers for Torus (T^2):")
    print(incremental_algorithm(torus_simplices()))


Betti Numbers for Circle (S^1):
[1, 1]

Betti Numbers for Sphere (S^2):
[1, 0, 1]

Betti Numbers for Torus (T^2):
[1, 2, 1]
